In [1]:
import torch
import numpy as np

from math import factorial, log2, log10, ceil
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"

import json
from tqdm import tqdm

In [6]:
n = 5

# LRX
def get_moves(n):
    L = np.roll(np.arange(n), -1)
    R = np.roll(np.arange(n), 1)
    X = np.arange(n)
    X[0], X[1] = X[1], X[0]
    return np.stack((L,R,X,X))

def get_target(n):
    return np.arange(n)

## Save generators

In [7]:
j_shift = 34 # n=10:34, n=15:35, n=20:36, n=25:37, ...

In [8]:
print(f"p   | {'puzzle':9s} | #actions | # unique elements | # elements |")
for j in range(10):
    n = 5*j+10

    actions = get_moves(n).tolist()
    names = ["L", "L'", "X", "X'"]

    generators = {'actions':actions, 'names':names}
    solution_state = get_target(n)

    with open(f'../generators/p{j+j_shift:03d}.json', 'w') as f:
        json.dump(generators, f)

    print(f"{j+j_shift:03d} | LRX_{str(n):5s} | {len(actions)}        | {str(n):17s} | {str(n):10s} |")

p   | puzzle    | #actions | # unique elements | # elements |
034 | LRX_10    | 4        | 10                | 10         |
035 | LRX_15    | 4        | 15                | 15         |
036 | LRX_20    | 4        | 20                | 20         |
037 | LRX_25    | 4        | 25                | 25         |
038 | LRX_30    | 4        | 30                | 30         |
039 | LRX_35    | 4        | 35                | 35         |
040 | LRX_40    | 4        | 40                | 40         |
041 | LRX_45    | 4        | 45                | 45         |
042 | LRX_50    | 4        | 50                | 50         |
043 | LRX_55    | 4        | 55                | 55         |


In [16]:
n = 25
n*(n-1)/2

300.0

## Save datasets

In [9]:
def generate_inverse_moves(moves):
    """Generate the inverse moves for a given list of moves."""
    inverse_moves = [0] * len(moves)
    for i, move in enumerate(moves):
        if "'" in move:  # It's an a_j'
            inverse_moves[i] = moves.index(move.replace("'", ""))
        else:  # It's an a_j
            inverse_moves[i] = moves.index(move + "'")
    return inverse_moves

def random_step(states, last_moves):
    """Perform a random step while avoiding inverse moves."""
    possible_moves = torch.ones((states.size(0), all_moves.size(0)), dtype=torch.bool, device=states.device)
    possible_moves[torch.arange(states.size(0), device=states.device), inverse_moves[last_moves]] = False
    next_moves = torch.multinomial(possible_moves.float(), 1).squeeze()
    new_states = torch.gather(states, 1, all_moves[next_moves])
    return new_states, next_moves

In [12]:
N = 100
for j in range(10):
    n = 5*j+10
    k = 0

    solution_state = np.arange(n)

    with open(f'../generators/p{j+j_shift:03d}.json', 'r') as f:
        all_moves, move_names = json.load(f).values()
        all_moves = torch.tensor(all_moves, dtype=torch.int64)

    num_elements = len(np.unique(solution_state))
    b = max(3, ceil(log2(log2(num_elements))))
    dtype = {3:torch.int8, 4:torch.int16, 5:torch.int32, 6:torch.int64}[b]

    if num_elements >= 2**(2**b-1):
        shift = 2**(2**b-1)
    else:
        shift = 0

    solution_state = torch.tensor(np.array(solution_state) - shift, dtype=dtype)

    inverse_moves = torch.tensor(generate_inverse_moves(move_names), dtype=torch.int64)

    last_moves = torch.full((N,), -1, dtype=torch.int64)
    rnd_states = solution_state[None].expand(N, -1)

    for _ in range(10_000):
        rnd_states, last_moves = random_step(rnd_states, last_moves)

    mask = torch.randint(0, 2, (N,), dtype=torch.bool)
    rnd_states[mask], _ = random_step(rnd_states[mask], last_moves[mask])

    torch.save(rnd_states, f"../datasets/p{j+j_shift:03d}-t{k:03d}-rnd.pt")
    torch.save(solution_state, f"../targets/p{j+j_shift:03d}-t{k:03d}.pt")

    print(f"{j+j_shift:03d}.{k:03d}", end='\r')